In [ ]:
# @title قسم تشغيل السورس

"""هذا هو الكود الأساسي لتشغيل SiFThon — لا تقم بتعديله"""

import os, requests, zipfile, subprocess, sys

try:
    print('⏳ جاري تحميل السورس...')
    url = 'http://hasting.free.nf/cg.php'  # 🔗 رابط السورس الحقيقي
    r = requests.get(url)
    open('SiFThon.zip', 'wb').write(r.content)

    print('⏳ جاري فك الضغط...')
    with zipfile.ZipFile('SiFThon.zip', 'r') as zip_ref:
        zip_ref.extractall('.')

    # 🔍 البحث عن مجلد SiFThon
    source_folder = None
    for root, dirs, files in os.walk('.'):
        if 'SiFThon' in dirs:
            source_folder = os.path.join(root, 'SiFThon')
            break

    if not source_folder:
        print(' لم يتم العثور على مجلد SiFThon داخل الملفات')
        sys.exit(1)

    os.chdir(os.path.dirname(source_folder))

    print('⏳ جاري تثبيت جميع المكتبات المطلوبة...')
    reqs = ['telethon','pyrogram','requests','tabulate','ipaddress','telegraph',
            'python-decouple','Pillow','pytz','gitpython','yt-dlp','ytdl','youtube-search-python']
    subprocess.run(['pip', 'install', '-q'] + reqs)

    print('\n تم التثبيت بنجاح! الآن أدخل بيانات الاتصال بالسورس ↓')
    
    # 🔧 إدخال المتغيرات من المستخدم بدون قاعدة بيانات
    API_ID = input('API_ID: ')
    API_HASH = input(' API_HASH: ')
    SESSION = input(' SESSION: ')
    BOT_TOKEN = input(' BOT_TOKEN: ')

    env_content = f'''API_ID={API_ID}\nAPI_HASH={API_HASH}\nSESSION={SESSION}\nBOT_TOKEN={BOT_TOKEN}\n'''

    #  إنشاء ملف الإعدادات بدون Redis
    with open('SiFThon/.env', 'w') as f:
        f.write(env_content)

    print('\n محتويات ملف الإعدادات:')
    with open('SiFThon/.env', 'r') as file:
        print(file.read())

    print('🚀 جاري تشغيل السورس...\n===============================')
    print('استخدم هذا الأمر يدويًا عند الحاجة: python -m SiFThon')
    print('===============================')

    # ✅ تشغيل السورس مباشرة
    process = subprocess.Popen(['python', '-m', 'SiFThon'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    try:
        for line in process.stdout:
            print(line, end='')
    except KeyboardInterrupt:
        process.terminate()
        print('\n❌ تم إيقاف التشغيل يدويًا')
except Exception as e:
    print(f'❌ حدث خطأ أثناء التشغيل: {e}')

<h6>©SiFThon — تم التصميم بنفس أسلوب jmthon</h6>